<a href="https://colab.research.google.com/github/Lord0fTheFlies/NLP-Neuromatch-Project/blob/main/Neuromatch_academy_project_classification_disease_LLM_vs_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
!pip install wandb
!pip install transformers[torch] --upgrade
!pip install accelerate --upgrade
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.8 MB/s eta 0:00:00


**Import the necessary modules**

In [ ]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import wandb
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [ ]:
ds = load_dataset("dhivyeshrk/Disease-Symptom-Extensive-Clean")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/246945 [00:00<?, ? examples/s]

In [ ]:
type(ds)

datasets.dataset_dict.DatasetDict

In [ ]:
ds.shape

{'train': (246945, 378)}

**Load and prepare the data for LLM and KNN separately**



In [ ]:
# # Read the CSV file into a DataFrame
# df = pd.read_csv("hf://datasets/dhivyeshrk/Disease-Symptom-Extensive-Clean/Final_Augmented_dataset_Diseases_and_Symptoms.csv")

# df = df.sort_values(by=df.columns[0])
# labels_list = []

# # Iterate over each row in the aggregated DataFrame
# for index, row in df.iterrows():
#     disease_name = row[df.columns[0]]  # Get the disease name
#     labels = []  # Temporary list to store labels for the current disease

#     # Iterate over the symptom columns (starting from index 1)
#     for col in df.columns[1:]:
#         if row[col] == 1:
#             labels.append(col)  # Append the column label if value is 1

#     # Concatenate the labels into a string separated by commas
#     labels_string = ", ".join(labels)
#     labels_list.append(labels_string)  # Append the concatenated string to the list

# # Add the concatenated labels as a new column in the aggregated DataFrame
# df['Symptom_labels'] = labels_list

# # Initialize a list of columns to keep
# columns_to_keep = ['diseases', 'Symptom_labels']

# # Drop all columns except 'disease' and 'Symptom_labels'
# LLM_df = df[columns_to_keep]
# KNN_df = df.drop(["Symptom_labels"], axis=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Reduce the size of the data (allow for only 50 entries per disease)**

In [ ]:
# import pandas as pd

# def filter_and_limit_diseases(df, min_rows=50, limit_rows=50):
#     # Group by the disease name and count the number of rows per disease
#     disease_counts = df.groupby('diseases').size()

#     # Identify diseases with at least min_rows entries
#     diseases_to_keep = disease_counts[disease_counts >= min_rows].index.tolist()

#     # Create a DataFrame with diseases that meet the min_rows criteria
#     filtered_df = df[df['diseases'].isin(diseases_to_keep)]

#     # Limit the number of rows for each disease to limit_rows
    # limited_df = filtered_df.groupby('diseases').apply(lambda x: x.sample(n=limit_rows, random_state=42)).reset_index(drop=True)

#     # Identify diseases that have been filtered out
#     diseases_filtered_out = disease_counts[disease_counts < min_rows].index.tolist()

#     # Calculate the average row count per disease after limiting
#     avg_row_count = limited_df.groupby('diseases').size().mean()

#     # Display the results
#     print(f"Diseases with at least {min_rows} entries: {len(diseases_to_keep)}")
#     print(f"Diseases filtered out ({len(diseases_filtered_out)}): {diseases_filtered_out}")
#     print(f"Average row count per disease after limiting: {avg_row_count:.2f}")

#     return limited_df, diseases_filtered_out

# # Use the function with your DataFrame
# KNN_filtered_limited_df, diseases_filtered_out = filter_and_limit_diseases(KNN_df)

# # Display the new DataFrame
# KNN_filtered_limited_df

Diseases with at least 50 entries: 527
Diseases filtered out (246): ['abscess of the lung', 'acanthosis nigricans', 'acariasis', 'acute fatty liver of pregnancy (aflp)', 'adrenal adenoma', 'adrenal cancer', 'allergy to animals', 'amblyopia', 'amyloidosis', 'anal fistula', 'anemia due to malignancy', 'anemia of chronic disease', 'aphakia', 'aspergillosis', 'atrophy of the corpus cavernosum', 'avascular necrosis', 'birth trauma', 'blepharospasm', 'bone spur of the calcaneous', 'breast cancer', 'breast cyst', 'bunion', 'carcinoid syndrome', 'cat scratch disease', 'central atherosclerosis', 'cervical disorder', 'cholesteatoma', 'chronic inflammatory demyelinating polyneuropathy (cidp)', 'chronic kidney disease', 'chronic ulcer', 'colorectal cancer', 'congenital heart defect', 'congenital malformation syndrome', 'connective tissue disorder', 'conversion disorder', 'cryptococcosis', 'cryptorchidism', 'cushing syndrome', 'cyst of the eyelid', 'cystic fibrosis', 'cysticercosis', 'decubitus ulc

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,abdominal aortic aneurysm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,abdominal aortic aneurysm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,abdominal aortic aneurysm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26345,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26346,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26347,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26348,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Implement mutual_info_classif**

In [ ]:
# X = KNN_filtered_limited_df.drop(columns=['diseases'])
# y = KNN_filtered_limited_df['diseases']

# # Check and convert data types
# # Ensure all features are numeric
# X = X.apply(pd.to_numeric)

# # Ensure the target variable is categorical
# y = y.astype('category')

# # Debug: Print data types to confirm
# print("Data types of features:")
# print(X.dtypes)
# print("Data type of target:")
# print(y.dtypes)

# # Apply mutual information feature selection
# # Select the top 10 features
# mi_selector = SelectKBest(mutual_info_classif, k=10)
# X_kbest = mi_selector.fit_transform(X, y)

# # Check if the selected features are empty
# if X_kbest.size == 0:
#     print("No features were selected. Check if the dataset is appropriate for feature selection.")
# else:
#     # Get the selected feature names
#     selected_features = X.columns[mi_selector.get_support()]

#     # Create a DataFrame with the selected features
#     X_reduced = pd.DataFrame(X_kbest, columns=selected_features)
#     KNN_df_feature_reduced = pd.concat([KNN_filtered_limited_df['diseases'], X_reduced], axis=1)

#     # Save the reduced DataFrame to a CSV file
#     KNN_df_feature_reduced.to_csv("KNN_df_10_feature_reduced.csv", index=False)
#     print("Feature selection completed and the DataFrame has been saved.")

Data types of features:
anxiety and nervousness             int64
depression                          int64
shortness of breath                 int64
depressive or psychotic symptoms    int64
sharp chest pain                    int64
                                    ...  
hip weakness                        int64
back swelling                       int64
ankle stiffness or tightness        int64
ankle weakness                      int64
neck weakness                       int64
Length: 377, dtype: object
Data type of target:
category
Feature selection completed and the DataFrame has been saved.


In [ ]:
# # Apply mutual information feature selection
# # Select the top 100 features
# mi_selector = SelectKBest(mutual_info_classif, k=100)
# X_kbest = mi_selector.fit_transform(X, y)

# # Check if the selected features are empty
# if X_kbest.size == 0:
#     print("No features were selected. Check if the dataset is appropriate for feature selection.")
# else:
#     # Get the selected feature names
#     selected_features = X.columns[mi_selector.get_support()]

#     # Create a DataFrame with the selected features
#     X_reduced = pd.DataFrame(X_kbest, columns=selected_features)
#     KNN_df_feature_reduced = pd.concat([KNN_filtered_limited_df['diseases'], X_reduced], axis=1)

#     # Save the reduced DataFrame to a CSV file
#     KNN_df_feature_reduced.to_csv("KNN_df_100_feature_reduced.csv", index=False)
#     print("Feature selection completed and the DataFrame has been saved.")

Feature selection completed and the DataFrame has been saved.


**Import the prepared datasets**

In [ ]:
KNN_df_100_feature_reduced = pd.read_csv("KNN_df_100_feature_reduced.csv")
LLM_df_stringify = pd.read_csv("LLM_df_stringify.csv").drop(LLM_df_stringify.columns[0], axis=1)

,diseases,Symptom_labels
0,abdominal aortic aneurysm,"shortness of breath, palpitations, sharp abdom..."
1,abdominal aortic aneurysm,"shortness of breath, palpitations, sharp abdom..."
2,abdominal aortic aneurysm,"palpitations, sharp abdominal pain, arm swelli..."
3,abdominal aortic aneurysm,"palpitations, arm swelling, burning abdominal ..."
4,abdominal aortic aneurysm,"palpitations, sharp abdominal pain, arm swelli..."
...,...,...
246940,zenker diverticulum,"sore throat, elbow weakness"
246941,zenker diverticulum,"sharp chest pain, sore throat, elbow weakness"
246942,zenker diverticulum,"sharp chest pain, sore throat, elbow weakness"
246943,zenker diverticulum,"sharp chest pain, sore throat, elbow weakness"


**Split the data into test, train, and validation sets.**

In [ ]:
# Convert the DataFrames to Hugging Face Datasets
LLM_dataset = Dataset.from_pandas(LLM_df_stringify)
KNN_dataset = Dataset.from_pandas(KNN_df_100_feature_reduced)

# Perform train-test splits for LLM dataset
llm_train_test_split = LLM_dataset.train_test_split(test_size=0.2)
llm_test_valid_split = llm_train_test_split['test'].train_test_split(test_size=0.5)

LLM_dataset_dict = DatasetDict({
    'train': llm_train_test_split['train'],
    'validation': llm_test_valid_split['train'],
    'test': llm_test_valid_split['test']
})

# Perform train-test splits for KNN dataset
knn_train_test_split = KNN_dataset.train_test_split(test_size=0.2)
knn_test_valid_split = knn_train_test_split['test'].train_test_split(test_size=0.5)

KNN_dataset_dict = DatasetDict({
    'train': knn_train_test_split['train'],
    'validation': knn_test_valid_split['train'],
    'test': knn_test_valid_split['test']
})

print(LLM_dataset_dict)
print(KNN_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['diseases', 'Symptom_labels'],
        num_rows: 197556
    })
    validation: Dataset({
        features: ['diseases', 'Symptom_labels'],
        num_rows: 24694
    })
    test: Dataset({
        features: ['diseases', 'Symptom_labels'],
        num_rows: 24695
    })
})
DatasetDict({
    train: Dataset({
        features: ['diseases', 'anxiety and nervousness', 'depression', 'shortness of breath', 'depressive or psychotic symptoms', 'sharp chest pain', 'dizziness', 'insomnia', 'abnormal involuntary movements', 'chest tightness', 'palpitations', 'irregular heartbeat', 'hoarse voice', 'sore throat', 'difficulty speaking', 'cough', 'nasal congestion', 'diminished hearing', 'difficulty in swallowing', 'skin swelling', 'retention of urine', 'leg pain', 'hip pain', 'suprapubic pain', 'blood in stool', 'lack of growth', 'abusing alcohol', 'fainting', 'hostile behavior', 'drug abuse', 'sharp abdominal pain', 'vomiting', 'headache', 'naus

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used for training.")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

GPU is not available, using CPU instead.


In [ ]:
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=25)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## tokenizeing the datasets

In [ ]:
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

def preprocess_function(examples):
    # Concatenate text and label to form the input for the language model
    examples["text"] = examples["text"] + " " + examples["label"]
    return examples


Map:   0%|          | 0/5328 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/667 [00:00<?, ? examples/s]

Map:   0%|          | 0/666 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to=["tensorboard", "wandb"],
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
# API 6a04f731cabc5d8f92eae9ae4c0f9d4457581828

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.519300,0.413661,0.950450,0.948429,0.955179,0.950450
2,0.157500,0.077186,0.989489,0.989430,0.990149,0.989489
3,0.053300,0.045084,0.996997,0.996982,0.997095,0.996997


TrainOutput(global_step=999, training_loss=0.6861919112451322, metrics={'train_runtime': 8767.3327, 'train_samples_per_second': 1.823, 'train_steps_per_second': 0.114, 'total_flos': 336843475056288.0, 'train_loss': 0.6861919112451322, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 0.059013482183218, 'eval_accuracy': 0.9940029985007496, 'eval_f1': 0.9939953276231427, 'eval_precision': 0.9943270666719427, 'eval_recall': 0.9940029985007496, 'eval_runtime': 87.8027, 'eval_samples_per_second': 7.597, 'eval_steps_per_second': 0.478, 'epoch': 3.0}


In [ ]:
wandb.finish()

eval/accuracy,▁▇██
eval/f1,▁▇██
eval/loss,█▂▁▁
eval/precision,▁▇██
eval/recall,▁▇██
eval/runtime,▁▁█▆
eval/samples_per_second,██▁▃
eval/steps_per_second,██▁▃
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▃▄▄▄▅▄▅▅▅▃▄▃▃▆▃▄▃▄▂▂▁▄▁▃▁▂▁▁▁▁▆▂▁▁█▂▁▁▁▄


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./logs